In [1]:
%pip install torch polars transformers tqdm rich sentencepiece protobuf

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import polars as pl
from transformers import AutoTokenizer, AutoModelForSequenceClassification
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)
LABEL_NAMES = ["ENTAILMENT", "NEUTRAL", "CONTRADICTION"]
input_file = "data/GPT-4o_raw_single_class_prompt.csv"
df = pl.read_csv(input_file)
print("Total samples", df.height)
df.columns

cpu
Total samples 29


/home/gioffre/projects/semantic_anli/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/gioffre/projects/semantic_anli/.conda/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


['cid',
 'premise',
 'hypothesis',
 'label',
 'Generated ENTAILMENT Hypothesis',
 'ENTAILMENT Correct?',
 'ENTAILMENT Adversarial?',
 'Generated NEUTRAL Hypothesis',
 'NEUTRAL Correct?',
 'NEUTRAL Adversarial',
 'Generated CONTRADICTION Hypothesis',
 'CONTRADICTION Correct?',
 'CONTRADICTION Adversarial']

# Compute the scores
Check whether the samples generated by LLama3 that are *correct* (i.e., checked by a human), are also *difficult* for the model (i.e., the model fails to classify them correctly).

This section only writes on another `.csv` file the results.

In [3]:
MODELS = {}
TOKENIZERS = {}

model_name_base = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
model_name_large = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
model_name_large_2 = "Joelzhang/deberta-v3-large-snli_mnli_fever_anli_R1_R2_R3-nli"
MAP = {
    'base' : model_name_base,
    'large1': model_name_large,
    'large2': model_name_large_2,
}

for model_name in MAP.values():
    print(f"> loading {model_name}")
    TOKENIZERS[model_name] = AutoTokenizer.from_pretrained(model_name, cache_dir='./.hf_cache')
    MODELS[model_name] = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir='./.hf_cache').to(device)


def inference(model_name, premise, hypothesis):
    model_input = TOKENIZERS[model_name](premise, hypothesis, truncation=False, return_tensors="pt")
    output = MODELS[model_name](model_input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    return {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, LABEL_NAMES)}

def predict(model_name, premise, hypothesis, label):
    prediction = inference(model_name, premise, hypothesis)
    predicted = max(prediction, key=prediction.get)
    return int(predicted != label)

> loading MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli
> loading MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli
> loading Joelzhang/deberta-v3-large-snli_mnli_fever_anli_R1_R2_R3-nli


/home/gioffre/projects/semantic_anli/.conda/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
from tqdm import tqdm
from pprint import pprint
res = {}
for label in LABEL_NAMES:
    for model_id in MAP.keys():
        res[f"new {label} hypothesis fools {model_id.upper()}"] = []
#pprint(res)

for model_id, model_name in MAP.items():
    for i, elem in tqdm(enumerate(df.iter_rows(named=True)), desc=f'> model {model_id}', total=df.height):
        for label in LABEL_NAMES:
            # filters empty cells
            if elem[f'Generated {label} Hypothesis'] == '':
                continue
            if elem[f'{label} Correct?'] == True:
                int_flag = predict(model_name, elem['premise'], elem['hypothesis'], elem['label'])
                res[f'new {label} hypothesis fools {model_id.upper()}'].append(int_flag)
            else:
                res[f'new {label} hypothesis fools {model_id.upper()}'].append(None)
#pprint(res)

> model large2: 100%|██████████| 29/29 [00:08<00:00,  3.46it/s]


In [11]:
print(res)

{'new ENTAILMENT hypothesis fools BASE': [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'new ENTAILMENT hypothesis fools LARGE1': [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'new ENTAILMENT hypothesis fools LARGE2': [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'new NEUTRAL hypothesis fools BASE': [None, None, None, None, None, None, None, None, None, None, 1, 0, 0, 0, 0, 0, 1, 0, 0, None, None, None, None, None, None, None, None, None, None], 'new NEUTRAL hypothesis fools LARGE1': [None, None, None, None, None, None, None, None, None, None, 1, 0, 0, 0, 0, 0, 1, 0, 0, None, None, None, None, None, None, None, None, None, None], 'new NEUTRAL hypothesis fools LARGE2': [None, None, None, None,

In [5]:
for label in LABEL_NAMES:
    for model_id in MAP.keys():
        df = df.with_columns(
            pl.Series(
                name=f"new {label} hypothesis fools {model_id.upper()}", 
                values=res[f"new {label} hypothesis fools {model_id.upper()}"]
            ),
        )
df.columns
df.head()

cid,premise,hypothesis,label,Generated ENTAILMENT Hypothesis,ENTAILMENT Correct?,ENTAILMENT Adversarial?,Generated NEUTRAL Hypothesis,NEUTRAL Correct?,NEUTRAL Adversarial,Generated CONTRADICTION Hypothesis,CONTRADICTION Correct?,CONTRADICTION Adversarial,new ENTAILMENT hypothesis fools BASE,new ENTAILMENT hypothesis fools LARGE1,new ENTAILMENT hypothesis fools LARGE2,new NEUTRAL hypothesis fools BASE,new NEUTRAL hypothesis fools LARGE1,new NEUTRAL hypothesis fools LARGE2,new CONTRADICTION hypothesis fools BASE,new CONTRADICTION hypothesis fools LARGE1,new CONTRADICTION hypothesis fools LARGE2
i64,str,str,str,str,bool,bool,str,bool,bool,str,bool,bool,i64,i64,i64,i64,i64,i64,i64,i64,i64
184052,"""Gangs of New York . The screen…","""Kenneth Lonergan is a writer o…","""ENTAILMENT""","""Hypothesis: Kenneth Lonergan, …",true,false,null,false,false,null,false,false,0,0,0,null,null,null,null,null,null
217178,"""Pakistan Movement . The Pakist…","""The Pakistan Movement was the …","""NEUTRAL""","""The series of social, politica…",true,false,null,false,false,null,false,false,0,0,0,null,null,null,null,null,null
169984,"""The Brat Pack is a nickname gi…","""The Brat Pack is a nickname gi…","""CONTRADICTION""","""In the 1980s, a group of young…",true,false,null,false,false,null,false,false,1,1,1,null,null,null,null,null,null
120315,"""Bret Easton Ellis . Ellis also…","""Bret Easton Ellis barely wrote…","""CONTRADICTION""","""Bret Easton Ellis, who penned …",true,false,null,false,false,null,false,false,1,1,1,null,null,null,null,null,null
145735,"""New York Knicks . The Knicks c…","""The New York Knicks are in the…","""ENTAILMENT""","""Competing in the National Bask…",true,false,null,false,false,null,false,false,0,0,0,null,null,null,null,null,null


In [6]:
new_df = df.with_columns(
    (
        pl.col("new ENTAILMENT hypothesis fools BASE")   +  
        pl.col("new ENTAILMENT hypothesis fools LARGE1") + 
        pl.col("new ENTAILMENT hypothesis fools LARGE2")
    ).alias("ENTAILMENT difficulty score"),
    (
        pl.col("new NEUTRAL hypothesis fools BASE")   +  
        pl.col("new NEUTRAL hypothesis fools LARGE1") + 
        pl.col("new NEUTRAL hypothesis fools LARGE2")
    ).alias("NEUTRAL difficulty score"),
    (
        pl.col("new CONTRADICTION hypothesis fools BASE")   +  
        pl.col("new CONTRADICTION hypothesis fools LARGE1") + 
        pl.col("new CONTRADICTION hypothesis fools LARGE2")
    ).alias("CONTRADICTION difficulty score"),
)
new_df.head()

cid,premise,hypothesis,label,Generated ENTAILMENT Hypothesis,ENTAILMENT Correct?,ENTAILMENT Adversarial?,Generated NEUTRAL Hypothesis,NEUTRAL Correct?,NEUTRAL Adversarial,Generated CONTRADICTION Hypothesis,CONTRADICTION Correct?,CONTRADICTION Adversarial,new ENTAILMENT hypothesis fools BASE,new ENTAILMENT hypothesis fools LARGE1,new ENTAILMENT hypothesis fools LARGE2,new NEUTRAL hypothesis fools BASE,new NEUTRAL hypothesis fools LARGE1,new NEUTRAL hypothesis fools LARGE2,new CONTRADICTION hypothesis fools BASE,new CONTRADICTION hypothesis fools LARGE1,new CONTRADICTION hypothesis fools LARGE2,ENTAILMENT difficulty score,NEUTRAL difficulty score,CONTRADICTION difficulty score
i64,str,str,str,str,bool,bool,str,bool,bool,str,bool,bool,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
184052,"""Gangs of New York . The screen…","""Kenneth Lonergan is a writer o…","""ENTAILMENT""","""Hypothesis: Kenneth Lonergan, …",true,false,null,false,false,null,false,false,0,0,0,null,null,null,null,null,null,0,null,null
217178,"""Pakistan Movement . The Pakist…","""The Pakistan Movement was the …","""NEUTRAL""","""The series of social, politica…",true,false,null,false,false,null,false,false,0,0,0,null,null,null,null,null,null,0,null,null
169984,"""The Brat Pack is a nickname gi…","""The Brat Pack is a nickname gi…","""CONTRADICTION""","""In the 1980s, a group of young…",true,false,null,false,false,null,false,false,1,1,1,null,null,null,null,null,null,3,null,null
120315,"""Bret Easton Ellis . Ellis also…","""Bret Easton Ellis barely wrote…","""CONTRADICTION""","""Bret Easton Ellis, who penned …",true,false,null,false,false,null,false,false,1,1,1,null,null,null,null,null,null,3,null,null
145735,"""New York Knicks . The Knicks c…","""The New York Knicks are in the…","""ENTAILMENT""","""Competing in the National Bask…",true,false,null,false,false,null,false,false,0,0,0,null,null,null,null,null,null,0,null,null


In [7]:
new_df.write_csv(input_file.replace('.csv', '_STATS.csv'))
print("> Written stats csv.")

> Written stats csv.


# Statistics
I want to see:
- how many generated examples are correct for each category
- how many correct examples are can fool 0, 1, 2, 3 models

In [8]:
from rich.console import Console
from rich.table import Table

console = Console()

new_df = pl.read_csv(input_file.replace('.csv', '_STATS.csv'))

In [14]:
table = Table(title=input_file.replace('.csv', '_STATS.csv'))

table.add_column("Category", justify="left", style="cyan", no_wrap=True)
table.add_column("Percentage", style="green", justify='center')
table.add_column("Values", justify="right", style="magenta")

for label in LABEL_NAMES:
    num_correct = new_df.filter(pl.col(f'{label} Correct?') == True).height
    support = new_df.filter(pl.col(f'Generated {label} Hypothesis') != '')
    table.add_row(
        f"Synthetic {label:<13} samples that are correct:", 
        f"{100*num_correct/support.height:.2f}%", 
        f"{num_correct}/{support.height}") 

console.print(table)
          

               data/GPT-4o_raw_single_class_prompt_STATS.csv               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ Category                                          ┃ Percentage ┃ Values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ Synthetic ENTAILMENT    samples that are correct: │  100.00%   │  10/10 │
│ Synthetic NEUTRAL       samples that are correct: │  100.00%   │    9/9 │
│ Synthetic CONTRADICTION samples that are correct: │  100.00%   │  10/10 │
└───────────────────────────────────────────────────┴────────────┴────────┘

In [15]:
table = Table(title=input_file.replace('.csv', '_STATS.csv'))

table.add_column("Category of correct synthetic samples", justify="left", style="cyan", no_wrap=False)
table.add_column("Fool 0 models", style="red", justify='center')
table.add_column("%", style="red", justify='center')
table.add_column("Fool >= 1 models", style="green", justify='center')
table.add_column("%", style="green", justify='center')
table.add_column("Fool >= 2 models", style="green", justify='center')
table.add_column("%", style="green", justify='center')
table.add_column("Fool >= 3 models", style="green", justify='center')
table.add_column("%", style="green", justify='center')
table.add_column("Support", justify="right", style="magenta")

for label in LABEL_NAMES:
    # retrieve correct samples per label
    correct_df = new_df.filter(pl.col(f'{label} Correct?') == True)
    num_correct = correct_df.height

    fool_0 = correct_df.filter(pl.col(f"{label} difficulty score") == 0).height
    fool_at_least_1 = correct_df.filter(pl.col(f"{label} difficulty score") >= 1).height
    fool_at_least_2 = correct_df.filter(pl.col(f"{label} difficulty score") >= 2).height
    fool_at_least_3 = correct_df.filter(pl.col(f"{label} difficulty score") >= 3).height

    table.add_row(
        label, 
        f"{fool_0:>2}",
        f"{100*fool_0/num_correct:.2f}%", 
        f"{fool_at_least_1:>2}",
        f"{100*fool_at_least_1/num_correct:.2f}%", 
        f"{fool_at_least_2:>2}",
        f"{100*fool_at_least_2/num_correct:.2f}%", 
        f"{fool_at_least_3:>2}",
        f"{100*fool_at_least_3/num_correct:.2f}%", 
        f"{num_correct}"
) 

console.print(table)

                                   data/GPT-4o_raw_single_class_prompt_STATS.csv                                   
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ Category of ┃             ┃        ┃            ┃        ┃             ┃        ┃            ┃        ┃         ┃
┃ correct     ┃             ┃        ┃            ┃        ┃             ┃        ┃            ┃        ┃         ┃
┃ synthetic   ┃   Fool 0    ┃        ┃ Fool >= 1  ┃        ┃  Fool >= 2  ┃        ┃ Fool >= 3  ┃        ┃         ┃
┃ samples     ┃   models    ┃   %    ┃   models   ┃   %    ┃   models    ┃   %    ┃   models   ┃   %    ┃ Support ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ ENTAILMENT  │      7      │ 70.00% │      3     │ 30.00% │      3      │ 30.00% │      3     │ 30.00% │      10 │
│ NEUTRAL     │      7      │ 77.78% │      2     │ 22.22% │      2      │ 22.22% │      2     │ 22.22% │       9 │
│ CONTRADICT… │      7      │ 70.00% │      3     │ 30.00% │      2      │ 20.00% │      1     │ 10.00% │      10 │
└─────────────┴─────────────┴────────┴────────────┴────────┴─────────────┴────────┴────────────┴────────┴─────────┘

export a csv containing only adversarial samples

In [17]:
to_df = {
    "cid" : [],
    "premise": [],
    "original hypothesis": [],
    "original label": [],
    "generated hypothesis": [],
    "prompted label": [],
    "models fooled": [],
}

for label in LABEL_NAMES:
    # retrieve correct samples per label
    correct_df = new_df.filter(pl.col(f'{label} Correct?') == True)
    num_correct = correct_df.height

    adversarial_samples = correct_df.filter(pl.col(f"{label} difficulty score") >= 1)
    for row in adversarial_samples.iter_rows(named=True):
        to_df['cid'].append(row['cid'])
        to_df['premise'].append(row['premise'])
        to_df["original hypothesis"].append(row['hypothesis'])
        to_df["original label"].append(row['label'])
        to_df["generated hypothesis"].append(row[f'Generated {label} Hypothesis'])
        to_df["prompted label"].append(label)
        to_df["models fooled"].append(row[f'{label} difficulty score'])
        
to_df = pl.DataFrame(to_df).write_csv(input_file.replace('.csv', 'ADV_SAMPLES.csv'))

stats on the token lenghts between original and correct generated hypothesis

In [25]:
from numpy import mean


original_hypotheses_len = [] 
generated_hypotheses_len = [] 

for label in LABEL_NAMES:
    samples = new_df.filter(
        (pl.col(f'{label} Correct?') == True) & (pl.col(f'Generated {label} Hypothesis') != '')
    )
    for row in samples.iter_rows(named=True):
        original_hypotheses_len.append(len(row['hypothesis'].strip().split(' ')))
        generated_hypotheses_len.append(len(row[f'Generated {label} Hypothesis'].strip().split(' ')))
print(original_hypotheses_len)
print(generated_hypotheses_len)
print(f"> original hypotheses mean length: {mean(original_hypotheses_len):.2f}")
print(f"> generated hypotheses mean length: {mean(generated_hypotheses_len):.2f}\t({mean(generated_hypotheses_len)/mean(original_hypotheses_len):.2f}x original)")

[7, 12, 13, 10, 9, 10, 11, 5, 7, 7, 7, 7, 8, 6, 5, 4, 9, 13, 9, 7, 9, 5, 9, 14, 6, 6, 6, 10, 7]
[32, 21, 21, 24, 21, 44, 25, 33, 48, 33, 22, 16, 22, 12, 23, 24, 24, 25, 23, 30, 30, 30, 29, 37, 24, 24, 28, 33, 33]
> original hypotheses mean length: 8.21
> generated hypotheses mean length: 27.28	(3.32x)
